### Basic Usage for data providers

In [5]:
import pandas as pd
import numpy as np
import logging

from pps_data_provider import PPSDataProvider
import pps_settings as settings

In [2]:
# Instantiate logger
logger = logging.getLogger(name="data_provider")

In [3]:
# Instantiate Provider
provider = PPSDataProvider(logger=logger)

In [9]:
# get list of tickers / start_date from the settings
list_of_tickers = settings.YF_TICK_LIST
start_dt = settings.DEFAULT_START_DT

price_df = provider.get_price_data(tickers=list_of_tickers, start_date=start_dt, end_date=None)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AMAZ']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [10]:
price_df

,DATE,FIELD,VALUE,TICKER
0,1995-01-03,Open,3.470980e-01,AAPL
1,1995-01-04,Open,3.448660e-01,AAPL
2,1995-01-05,Open,3.504460e-01,AAPL
3,1995-01-06,Open,3.716520e-01,AAPL
4,1995-01-09,Open,3.716520e-01,AAPL
...,...,...,...,...
103495,2024-03-27,Volume,1.670500e+07,MSFT
103496,2024-03-28,Volume,2.187120e+07,MSFT
103497,2024-04-01,Volume,1.631600e+07,MSFT
103498,2024-04-02,Volume,1.791200e+07,MSFT


### A few ways to manipulate data

#### 1.Pivoting DataFrames

In [14]:
# pivoting dataframe : use built in pivot_table. Arguments are self-explanatory.
# Look out for non-unique index (meaning, [TICKER, DATE] has to be unique - otherwise, pivot_table() has default aggregation set up)
# aggfunc (default is sum), fill_value (None : what to fill in if no value is found for the index/column combo)

pivoted_view = price_df.pivot_table(index=["TICKER", "DATE"], 
                                    columns="FIELD", 
                                    values="VALUE").reset_index()

In [16]:
pivoted_view.head(10)

FIELD,TICKER,DATE,Adj Close,Close,High,Low,Open,Volume
0,AAPL,1995-01-03,0.286800,0.342634,0.347098,0.338170,0.347098,1.038688e+08
1,AAPL,1995-01-04,0.294274,0.351563,0.353795,0.344866,0.344866,1.586816e+08
2,AAPL,1995-01-05,0.290536,0.347098,0.351563,0.345982,0.350446,7.364000e+07
3,AAPL,1995-01-06,0.313891,0.375000,0.385045,0.367188,0.371652,1.076622e+09
4,AAPL,1995-01-09,0.307936,0.367885,0.373884,0.366071,0.371652,2.740864e+08
5,AAPL,1995-01-10,0.326503,0.390067,0.392857,0.368304,0.368304,6.147904e+08
6,AAPL,1995-01-11,0.349391,0.417411,0.429129,0.381138,0.390625,8.738240e+08
7,AAPL,1995-01-12,0.339115,0.405134,0.414063,0.399554,0.411830,5.517792e+08
8,AAPL,1995-01-13,0.335378,0.400670,0.411830,0.396205,0.411830,3.513776e+08
9,AAPL,1995-01-16,0.332575,0.397321,0.404018,0.395089,0.400670,1.889776e+08


#### 2. Undoing pivot (stacking) 

In [29]:
# Melt stacks the data -- compare with the pivot_table from above
stacked_view = pivoted_view.melt(id_vars=["TICKER","DATE"], 
                                 value_vars=["Adj Close", "Close", "High", "Low", "Open", "Volume"],
                                 var_name="FIELD",
                                 value_name="VALUE")

In [31]:
stacked_view.head(10)

,TICKER,DATE,FIELD,VALUE
0,AAPL,1995-01-03,Adj Close,0.286800
1,AAPL,1995-01-04,Adj Close,0.294274
2,AAPL,1995-01-05,Adj Close,0.290536
3,AAPL,1995-01-06,Adj Close,0.313891
4,AAPL,1995-01-09,Adj Close,0.307936
5,AAPL,1995-01-10,Adj Close,0.326503
6,AAPL,1995-01-11,Adj Close,0.349391
7,AAPL,1995-01-12,Adj Close,0.339115
8,AAPL,1995-01-13,Adj Close,0.335378
9,AAPL,1995-01-16,Adj Close,0.332575


#### 3. Groupby and its usage


In [32]:
# Group by is one of the most important built in when dealing with dataframes. 
# Groupby works the best with normalized table format. (Single value column, rest are ID columns)

In [37]:
# Groupby can either return a dataframe or series (based on as_index=True/False)

# Finding Max VALUE FOR EACH FIELD / FOR EACH MONTH:
# First, we need to understand the dataframe itself, what the datatype is for each column 
price_df.dtypes

# In pandas dataframe, each column is pd.Series - there is no separate "string" datatype for series
# Any non-numeric or non-bool (or some other types) are classified as object - as you can see dtype of FIELD is object, not string


DATE      datetime64[ns]
FIELD             object
VALUE            float64
TICKER            object
dtype: object

In [38]:
# always good to check for any null values
price_df.isnull().any()

DATE      False
FIELD     False
VALUE     False
TICKER    False
dtype: bool

In [40]:
# Now since we know DATE column is already a datetime type, no need to convert. If Date column is a string (object) type,
# it is always a good idea to convert it to datetime format because that gives access to many built in functions

In [42]:
price_df["DATE"] = pd.to_datetime(price_df["DATE"])

In [43]:
# Now lets find MAX value for each field / for each month
# there are several ways to do it 

In [76]:
max_df = price_df.set_index("DATE").groupby(["FIELD", "TICKER"]).resample("M")["VALUE"].max().reset_index()

In [77]:
# Resample, resamples the date (need datetime column to be the index) - returning one row per resample frequency
# we need to specify which operations to do for each frequency : .max(), .first(), .last(), .mean()
max_df

,FIELD,TICKER,DATE,VALUE
0,Adj Close,AAPL,1995-01-31,3.493913e-01
1,Adj Close,AAPL,1995-02-28,3.278693e-01
2,Adj Close,AAPL,1995-03-31,3.016396e-01
3,Adj Close,AAPL,1995-04-30,2.932084e-01
4,Adj Close,AAPL,1995-05-31,3.306793e-01
...,...,...,...,...
4951,Volume,MSFT,2023-12-31,7.847820e+07
4952,Volume,MSFT,2024-01-31,4.787110e+07
4953,Volume,MSFT,2024-02-29,3.194730e+07
4954,Volume,MSFT,2024-03-31,4.504980e+07


Let's check if above is correct

In [100]:
price_df[(price_df["DATE"]>=pd.to_datetime("2024-01-01")) & 
         (price_df["DATE"]<=pd.to_datetime("2024-01-31")) & 
         (price_df["FIELD"]=="Volume") &
         (price_df["TICKER"]=="MSFT")]["VALUE"].max()


47871100.0

In [105]:
# Another way to do it is grouping by MONTHEND DATE
from pandas.tseries.offsets import MonthEnd

def nth_monthend_date(date_col, n):
    new_date_col = date_col + MonthEnd(n) 
    date_col = new_date_col.where(new_date_col.dt.month==date_col.dt.month,
                                  date_col)
    return date_col

price_df_copy = price_df.copy()
price_df_copy["EOM_DATE"] = nth_monthend_date(price_df_copy["DATE"], 1)
    
# You can see now we have end of month dates in EOM_DATE column

In [106]:
price_df_copy

,DATE,FIELD,VALUE,TICKER,EOM_DATE
0,1995-01-03,Open,3.470980e-01,AAPL,1995-01-31
1,1995-01-04,Open,3.448660e-01,AAPL,1995-01-31
2,1995-01-05,Open,3.504460e-01,AAPL,1995-01-31
3,1995-01-06,Open,3.716520e-01,AAPL,1995-01-31
4,1995-01-09,Open,3.716520e-01,AAPL,1995-01-31
...,...,...,...,...,...
103495,2024-03-27,Volume,1.670500e+07,MSFT,2024-03-31
103496,2024-03-28,Volume,2.187120e+07,MSFT,2024-03-31
103497,2024-04-01,Volume,1.631600e+07,MSFT,2024-04-30
103498,2024-04-02,Volume,1.791200e+07,MSFT,2024-04-30


In [107]:
# Now we can use this new column for groupby
max_df_1 = price_df_copy.groupby(["EOM_DATE", "TICKER", "FIELD"], as_index=False)["VALUE"].max()

In [113]:
# now let's compare the two dfs
# Another useful built in function is merge - merge is used to join two dataframe. 
# usually, we don't do right_on, left_on since it will cretae dup columns (unless that's what you want)
# We usually rename column names so that they match. -- another way of doing below
# max_df = max_df.rename(columns={"DATE" : "EOM_DATE"})
# merged = max_df.merge(max_df_1, on=["EOM_DATE", "TICKER", "FIELD"], how="outer", "suffixes=("_method_1", "method_2")") 
# If you don't specify suffixes, then any columns that are not in join index ("on") will have default suffix of _x, _y

merged = max_df.merge(max_df_1, right_on=["EOM_DATE", "TICKER", "FIELD"], left_on=["DATE", "TICKER", "FIELD"], how="outer")

In [114]:
abs(merged["VALUE_x"] - merged["VALUE_y"]).sum()

0.0